# Prepare Notebook
* install and import packages
* set up environment variables

In [6]:
pip install pandas dotenv "psycopg[binary]" openai neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 145.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 147.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 216.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 289.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [openai] 9/10 [openai]]
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import psycopg                      # SQL query packag}e
import os                           # get env variables
from urllib.request import urlopen  # package for HTTP connections
import time

from openai import OpenAI
import json
import csv
from typing import List, Dict, Any, Optional, Union

from neo4j import GraphDatabase

from dotenv import load_dotenv     # environment variables

## Set Up Environment Variables
To see variables: (1) `cat .env` in terminal OR (2) `%env` in notebook

**To make changes to .env file:**
1. In terminal: `cat .env` to see all variables
2. Create new text file
3. Paste all variables to be kept in new .env file
4. Save as any name (ex: test.txt)
5. In terminal: `mv test.txt .env` to rename 'test.txt' as '.env'

In [3]:
load_dotenv("/home/jovyan/work/Group-3/.env")

True

# Get Emails from Postgres and Solr

## Postgres (WIP)
* establish database connection
* query
* store query results
* close connection

In [ ]:
# REMEMBER TO CLOSE CONNECTION

try:
    conn = psycopg.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=int(os.getenv("DB_PORT",5432))
    )
    print('connected!')
except Exception as e:
    print(e)

In [5]:
# get IDs of email documents

query = """
SELECT id
FROM ucsf_opioid."Kentucky_Opioid_Litigation_Documents"
WHERE LOWER(type) = 'email'
UNION ALL
SELECT id
FROM ucsf_opioid."National_Prescription_Opiate_Litigation_Documents"
WHERE LOWER(type) = 'email'
UNION ALL
SELECT id
FROM ucsf_opioid."Oklahoma_Opioid_Litigation_Documents"
WHERE LOWER(type) = 'email'
UNION ALL
SELECT id
FROM ucsf_opioid."KHN_OxyContin_Collection"
WHERE LOWER(type) = 'email';
"""

In [6]:
cursor = conn.cursor('server_cursor')

In [7]:
cursor.execute(query)

<psycopg.ServerCursor 'server_cursor' [COMMAND_OK] [INTRANS] (host=awesome-compute.sdsc.edu user=habaek database=postgres) at 0x7fa7c2877a50>

In [8]:
records = cursor.fetchall()
print(len(records))

652


In [9]:
conn.close()

In [10]:
records[:5]

[('htcf0232',), ('sgdf0232',), ('rhdf0232',), ('njdf0232',), ('pndf0232',)]

In [11]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 226.2 MB/s eta 0:00:00


In [12]:
from sqlalchemy import create_engine,text

db_path = f"postgresql+psycopg2://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{int(os.getenv("DB_PORT",5432))}/{os.getenv("DB_NAME")}"
engine = create_engine(db_path)

# plsql block
plsql_query = """
DO $$
DECLARE
    table_name TEXT;
    table_array TEXT[] := ARRAY['ucsf_opioid."Kentucky_Opioid_Litigation_Documents"', 'ucsf_opioid."National_Prescription_Opiate_Litigation_Documents"', 'ucsf_opioid."Oklahoma_Opioid_Litigation_Documents"', 'ucsf_opioid."KHN_OxyContin_Collection"'];
BEGIN
    CREATE TEMP TABLE IF NOT EXISTS temp_all_ids (id TEXT) ON COMMIT PRESERVE ROWS;
    TRUNCATE temp_all_ids;
    
    FOREACH table_name IN ARRAY table_array LOOP
        EXECUTE format('INSERT INTO temp_all_ids SELECT id FROM %s WHERE lower(type) = ''email''', table_name);
    END LOOP;
END $$;
"""

with engine.connect() as conn:
    # Execute DO block
    conn.execute(text(plsql_query))
    conn.commit()
    
    # Fetch all IDs from temp table
    records = conn.execute(text("SELECT id FROM temp_all_ids"))
    records_list = [row[0] for row in records]
    
print(f"Number of rows: {records_list}")

OperationalError: (psycopg2.OperationalError) could not translate host name "None" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [11]:
list_of_ids = []

for i in records:
    list_of_ids.append(i[0])
list_of_ids

['htcf0232',
 'sgdf0232',
 'rhdf0232',
 'njdf0232',
 'pndf0232',
 'frcf0232',
 'mrlw0232',
 'krlw0232',
 'lrlw0232',
 'nrlw0232',
 'yrlw0232',
 'prlw0232',
 'qrlw0232',
 'srlw0232',
 'rrlw0232',
 'trlw0232',
 'zrlw0232',
 'fslw0232',
 'xslw0232',
 'gslw0232',
 'hslw0232',
 'jslw0232',
 'kslw0232',
 'lslw0232',
 'mslw0232',
 'nslw0232',
 'yslw0232',
 'pslw0232',
 'qslw0232',
 'tslw0232',
 'rslw0232',
 'sslw0232',
 'zslw0232',
 'htlw0232',
 'ktlw0232',
 'ytlw0232',
 'hllw0232',
 'jllw0232',
 'lllw0232',
 'nllw0232',
 'nnlw0232',
 'pllw0232',
 'rllw0232',
 'tllw0232',
 'hmlw0232',
 'gmlw0232',
 'jmlw0232',
 'lmlw0232',
 'nmlw0232',
 'ymlw0232',
 'zmlw0232',
 'rmlw0232',
 'fnlw0232',
 'xnlw0232',
 'jnlw0232',
 'lnlw0232',
 'pnlw0232',
 'rnlw0232',
 'tnlw0232',
 'snlw0232',
 'gylw0232',
 'hylw0232',
 'nylw0232',
 'qylw0232',
 'pylw0232',
 'sylw0232',
 'zylw0232',
 'gplw0232',
 'jplw0232',
 'lplw0232',
 'mplw0232',
 'nplw0232',
 'yplw0232',
 'pplw0232',
 'rplw0232',
 'qplw0232',
 'splw0232',

## Solr
creates `emails_list` = [{"email_id": email_id, "email_text": email_text}]

In [13]:
# add the email body from Solr to the metadata
emails_list = []
start_time = time.time()
for i in list_of_ids:
    id_and_body = {}
    record_id = i

    id_and_body["email_id"] = record_id

    solr_url = f'http://awesome-compute.sdsc.edu:8990/solr/ucsf-opioid/select?indent=true&q.op=OR&q=id%3A{record_id}&useParams=&wt=json'
    connection = urlopen(solr_url)

    response_text = connection.read().decode('utf-8')
    data = json.loads(response_text)
    email_body = data['response']['docs'][0]['ocr_text'][0]
    
    id_and_body["email_text"] = email_body
    emails_list.append(id_and_body)
print("execution time (s):", time.time() - start_time)

execution time (s): 120.22524619102478


In [14]:
emails_list[0]

{'email_id': 'htcf0232',
 'email_text': 'Unknown Fcom:    Sent:    To:    Ce:    Subject    i iol\'!tf.:\'iiWJffsaoffl*w�    Or .Richard Sacljler    Wednesday, September 04.1&96 9:19 AM    Friedman    Alfonso    Re{3].- Press release or similar promotion    THOSE -WILL  BE GOOD  ENOUGH  FOR  KE    Reply Separator    Subject;  Re 173: .Press release or similar.prntsotion Author:     Pri  dman at NORWALK pate:          9-4-96  7:39 AM    It will not  be circulated until  it m    ts my standards. KF    Reply Separator    Subject:   Re[6]:   Press  release or  similar promotion Author:    .Dr .Richard Sackler .at NORWALK Date:          9/4/.S6   6:26  AM    great but who will do  it?    EdC?     If so.,  this is  fine,   but I dcr.\'t  want   to be  involved until   it rase to your  standards.    Reply Separator    �Subject:  Re IS].: \'Preso\'release-or similar promotion Author:     Friedman  at NORWALK Date:          S-3-96   10:32  PM    We can do  this   in-bouse.   If you agree,   I\'

# Pass Emails Through LLM
https://platform.openai.com/docs/api-reference/chat/create

https://platform.openai.com/docs/pricing

https://platform.openai.com/usage

In [17]:
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

In [18]:
# testing LLM connection by running small query 
LLM_response = client.chat.completions.create(
    model="gpt-5-nano",
#    temperature=0.2,  # lower temperature = more focused, deterministic, and predictable text
    messages = (
        {
            'role': 'developer',
            'content': 'You are a helpful assistant.'
        },
        {
            'role': 'user',
            'content': 'tell me an anti-joke'
        }
    )
  )
    
output = LLM_response.choices[0].message.content
output

'I can’t fetch live weather data in this chat. But I can help you get it quickly or interpret a report. Here are easy options:\n\n- Check your phone or smart speaker: “What’s the weather in San Diego?”\n- Quick web check: search “San Diego weather now” or visit weather.com, accuweather.com, or the National Weather Service at weather.gov and look for San Diego.\n- If you tell me a specific neighborhood (Downtown, La Jolla, etc.), I can explain what a typical current report would look like for that area and how to read it (temperature, feels like, wind, humidity, precipitation chances, sky conditions).\n\nWould you like me to guide you to a specific source or help interpret a weather report you already have?'

## schema 
stores in `schema_json`

In [23]:
schema = {
  "@context": {
    "@vocab": "https://schema.org/",
    "email": "https://schema.org/EmailMessage",
    "person": "https://schema.org/Person",
    "org": "https://schema.org/Organization",
    "document": "https://schema.org/DigitalDocument",
    "topicEntity": "https://schema.org/Thing",
    "url": "https://schema.org/URL",
    "gpe": "https://schema.org/Place",
    "drug": "https://schema.org/Drug"
  },

  "@type": "case:Legislation",
  "semantic_type": "Legal Communication Record",
  "identifier": "Case-2005-Folio-185",
  "legalStatus": "Confidential document disclosed in litigation",
  "dateFiled": "2005-01-01",
  "language": ["en"],

  "confidentialityNotice": "Confidential document disclosed in litigation: Gallaher International Limited v. Tlais Enterprises Limited (2005 Folio 185).",

  "hasPart": {
    "@type": "email:EmailMessage",
    "semantic_type": "Email Communication",
    "identifier": "Email-20040908-1816-Giudice",
    "subject": "RE: Tlais costings final",
    "dateSent": "2004-09-08T18:16:00+02:00",
    "importance": "High",

    "sender": {
      "@type": "person:Person",
      "semantic_type": "Person",
      "name": "Linda Giudice",
      "email": "lgiudice@gallaherswitzerland.ch",
      "affiliation": {
        "@type": "org:Organization",
        "semantic_type": "ORG",
        "name": "Gallaher Switzerland SA",
        "role": "Subsidiary",
        "parentOrganization": {
          "@type": "org:Organization",
          "semantic_type": "ORG",
          "name": "Gallaher International Limited",
          "role": "Parent Corporation"
        }
      }
    },

    "recipient": [
      {
        "@type": "person:Person",
        "semantic_type": "Person",
        "name": "Jonathan Wale",
        "email": "JWale@gallaherswitzerland.ch",
        "affiliation": {
          "@type": "org:Organization",
          "semantic_type": "ORG",
          "name": "Gallaher Switzerland SA",
          "role": "Subsidiary"
        }
      }
    ],

    "body": "To discuss – I think this is on a restricted list.",

    "mentions": [
      {
        "@type": "topicEntity",
        "semantic_type": "Legal Case",
        "name": "Gallaher International Limited v. Tlais Enterprises Limited",
        "identifier": "2005 Folio 185"
      },
      {
        "@type": "gpe",
        "semantic_type": "GPE",
        "role": "Geographic Destination",
        "name": "Dubai",
        "identifier": "Wikidata:Q613"
      },
      {
        "@type": "gpe",
        "semantic_type": "GPE",
        "role": "Geographic Destination",
        "name": "Iraq",
        "identifier": "Wikidata:Q796"
      },
      {
        "@type": "topicEntity",
        "semantic_type": "Product Brand",
        "name": "Dorchester Slims"
      },
      {
        "@type": "topicEntity",
        "semantic_type": "Business Operation",
        "name": "Freight Logistics"
      },
      {
        "@type": "topicEntity",
        "semantic_type": "Financial Document",
        "name": "Tlais costings final"
      },
      {
        "@type": "topicEntity",
        "semantic_type": "Drug Name",
        "name": "Oxycontin"
      },
      {
        "@type": "topicEntity",
        "semantic_type": "Drug Name",
        "name": "Hydrocodone"
      }

    ],

    "attachments": [
      {
        "@type": "document:DigitalDocument",
        "semantic_type": "Spreadsheet Document",
        "name": "TLAIS costings 8.09.04.xls",
        "fileFormat": "application/vnd.ms-excel",
        "description": "Spreadsheet detailing Dorchester Slims costing, freight rates, and origination costs from Poland."
      },
      {
        "@type": "document:DigitalDocument",
        "semantic_type": "Presentation Document",
        "name": "Tlais Presentation.ppt",
        "fileFormat": "application/vnd.ms-powerpoint",

        "description": "Internal presentation outlining Tlais brand positioning and shipment forecasts."
      },
      {
        "@type": "document:DigitalDocument",
        "semantic_type": "Policy Document",
        "name": "Restricted Shipping Policy.pdf",
        "fileFormat": "application/pdf",
        "description": "PDF outlining Gallaher’s restricted destinations and compliance obligations for Middle East freight."
      }
    ],

    "forwardedMessage": {
      "@type": "email:EmailMessage",
      "semantic_type": "Forwarded Email Communication",
      "subject": "FW: Tlais costings final",
      "dateSent": "2004-09-08T17:36:00+02:00",
      "importance": "High",
      "sender": {
        "@type": "person:Person",
        "semantic_type": "Person",
        "name": "Nataliya Davydova",
        "email": "ndavydova@gallaherswitzerland.ch",
        "affiliation": {
          "@type": "org:Organization",
          "semantic_type": "ORG",
          "name": "Gallaher Switzerland SA"
        }
      },
      "recipient": [
        { "@type": "person:Person", "semantic_type": "Person", "name": "Jonathan Wale" },
        { "@type": "person:Person", "semantic_type": "Person", "name": "Paul Murden" },
        { "@type": "person:Person", "semantic_type": "Person", "name": "Stewart Hainsworth" }
      ],
      "body": "- Dorchester Slims – cost per 1,000 units from Poland\n- Freight for Iraq – rate to Dubai (agreed between Paul and Norman)\n- Origination costs per brand, as projected by Poland.",
      "attachments": [
        {
          "@type": "document:DigitalDocument",
          "semantic_type": "Spreadsheet Document",
          "name": "TLAIS costings 8.09.04.xls"
        }
      ]
    },

    "mentionsEmail": [
      {
        "@type": "email:EmailMessage",
        "semantic_type": "Referenced Email",
        "identifier": "Email-<REFERENCE-ID>",
        "relationType": "semantic-mention"
      }
    ],


    "structuredArgument": [
      {
        "semantic_type": "Internal Business Note",
        "claim": "Potential shipment from Dubai to Iraq may be subject to restricted list control.",
        "evidence": "Email chain notes freight to Dubai for Iraq with high importance flag.",
        "proposedAction": "Review export compliance and confirm if restricted destination list applies."
      }
    ],

    "complianceContext": {
      "@type": "CreativeWork",
      "semantic_type": "Regulatory and Legal Framework",
      "name": "Export Control and Restricted Destination Compliance",
      "keywords": [
        "Restricted list",
        "Freight logistics",
        "Export compliance",
        "Litigation disclosure"
      ],
      "about": [
        "Internal cost communication within tobacco trade",
        "Possible export compliance issue under restricted list regulations"
      ]
    }
  }
}

schema_json = json.dumps(schema, indent=2)

## continue

In [32]:
system_instructions = f"""You are specialized information extraction system reading emails.
TASKS:
- Read in the data from the user input
- Fix obvious OCR artifacts.
- Uncouple the email thread and identify email elements in reverse-chronological order. 
- Identify entities in the data and extract the name and semantic type. 
- Fix obvious OCR artifacts.
- If you find entity types not present in the schema, append them to the "mentions" list. 
RULES:
- Preserve the original semantics and legal meaning.
- Do not hallucinate content that is not inferable from the text.
- Output strictly must be a JSON only (no markdown, no prose, no commentary).
- Use brief, faithful witness spans copied from the input text. 
- Do NOT fabricate entities; rely only on the given narrative.
- Follow the output JSON in this schema {schema_json}
"""

## Batch processing functions
chunk, build, submit, wait, output file

In [ ]:
# breaks input into chunks before passing through batch build function
def chunk(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]

In [ ]:
"""
Build a JSONL file where each line is a Batch request for /v1/chat/completions,
keyed by email_id via custom_id.
"""
def build_batch_jsonl(
    emails: List[Dict[str, str]],
    system_instructions: str,
    jsonl_path: str,
    model: str = "gpt-4o-mini",  # or any chat model with JSON mode
) -> None:
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for rec in emails:
            email_id = rec["email_id"]
            email_text = rec["email_text"]

            task = {
                "custom_id": str(email_id),
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": model,
                    "response_format": {"type": "json_object"},
                    # "temperature": 0.0,
                    "max_tokens": 2000,
                    "messages": [
                        {
                            "role": "system",
                            "content": system_instructions,
                        },
                        {
                            "role": "user",
                            "content": email_text + "Uncouple the email thread and identify email elements in reverse-chronological order.",
                        },
                    ],
                },
            }

            f.write(json.dumps(task, ensure_ascii=False) + "\n")

In [ ]:
"""
Upload the batch JSONL file and create a Batch job.
Returns the batch_id.
"""
def submit_batch(jsonl_path: str) -> str:
    # 1. Upload file
    with open(jsonl_path, "rb") as f:
        batch_file = client.files.create(
            file=f,
            purpose="batch",
        )

    # 2. Create batch job
    batch = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",  # currently the only allowed window
        metadata={"job_type": "ocr_email_structuring"},
    )

    print("Created batch:", batch.id)
    return batch.id

In [ ]:
"""
Polls the batch until it reaches a terminal state or until max_polls is exceeded.
Returns the last seen batch object.
"""
def wait_for_batch(batch_id: str, poll_interval: int = 60, max_polls: int = 120):
    polls = 0
    last_completed = None

    while True:
        batch = client.batches.retrieve(batch_id)
        polls += 1

        print(
            f"Batch {batch_id} status: {batch.status} | "
            f"completed={batch.request_counts.completed} / total={batch.request_counts.total}"
        )

        # Track progress; if you want, detect "stuck" here
        if last_completed is None or batch.request_counts.completed != last_completed:
            last_completed = batch.request_counts.completed

        if batch.status in ("completed", "failed", "expired", "cancelled"):
            return batch

        if polls >= max_polls:
            print(f"Reached max_polls={max_polls}, stopping wait loop.")
            return batch

        time.sleep(poll_interval)

In [ ]:
"""
Given a completed batch object, download its output JSONL and return:
    { custom_id (email_id): parsed_json }

If output_jsonl_path is given, append each as:
    {"email_id": "<id>", "data": <parsed_json>}
to that JSONL file.
"""
def download_and_parse_results(
    batch,
    output_jsonl_path: Optional[str] = None,
) -> Dict[str, Any]:
    if not batch.output_file_id:
        raise RuntimeError(f"Batch {batch.id} has no output_file_id; status={batch.status}")

    file_content = client.files.content(batch.output_file_id).text

    results_by_email_id: Dict[str, Any] = {}

    out_f = open(output_jsonl_path, "a", encoding="utf-8") if output_jsonl_path else None

    try:
        for line in file_content.splitlines():
            if not line.strip():
                continue

            record = json.loads(line)
            custom_id = record["custom_id"]
            body = record["response"]["body"]

            message = body["choices"][0]["message"]
            content = message["content"]

            # We asked for JSON-only output
            try:
                parsed_json = json.loads(content)
            except json.JSONDecodeError:
                parsed_json = {"_raw_content": content}

            results_by_email_id[custom_id] = parsed_json

            if out_f is not None:
                normalized_line = {
                    "email_id": custom_id,
                    "output": parsed_json,
                }
                out_f.write(json.dumps(normalized_line, ensure_ascii=False) + "\n")
    finally:
        if out_f is not None:
            out_f.close()

    return results_by_email_id

## execute batch
takes about 30 minutes, about $0.72 for 652 emails

In [ ]:
batch_ids = []
for i, email_chunk in enumerate(chunk(emails_list, 300)):
    jsonl_path = f"Hayley/batch_input_{i:04d}.jsonl"
    build_batch_jsonl(email_chunk, system_instructions, jsonl_path)
    batch_id = submit_batch(jsonl_path)
    batch_ids.append(batch_id)
    final_batch = wait_for_batch(batch_id, poll_interval=60)

In [ ]:
combined_jsonl = "Hayley/all_emails_structured.jsonl"

for batch_id in batch_ids:
    batch = client.batches.retrieve(batch_id)
    if batch.status != "completed":
        print(f"Skipping batch {batch_id} with status={batch.status}")
        continue
        
    download_and_parse_results(batch, output_jsonl_path=combined_jsonl)

print(f"Combined output written to {combined_jsonl}")